In [14]:
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz 
import os
import pickle
from matplotlib.font_manager import FontProperties
sns.set(font=['sans-serif'])
sns.set_style("whitegrid",{"font.sans-serif":['Microsoft JhengHei']})
# scikit-learn
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer, RobustScaler
from sklearn import datasets, linear_model, preprocessing, neighbors
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix,mean_absolute_error,mean_squared_error, r2_score
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,KFold,RandomizedSearchCV
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
import xgboost as xgb

In [2]:
df_oh = pd.read_csv('../高雄市_predictset_onehot.csv', engine='python',index_col=[0])
df_oh

,交易年,總樓層數,主要建材,有無管理組織,建物移轉總面積坪,每坪價格,車位,屋齡,房間數,樓層比,...,交易標的_房地(土地+建物),交易標的_房地(土地+建物)+車位,都市土地使用分區_住,都市土地使用分區_其他,都市土地使用分區_商,建物型態_住宅大樓(11層含以上有電梯),建物型態_公寓(5樓含以下無電梯),建物型態_套房(1房1廳1衛),建物型態_華廈(10層含以下有電梯),建物型態_透天厝
0,7,14,4,1,24.614425,20.232039,0,16,4,0.500000,...,1,0,1,0,0,1,0,0,0,0
1,7,15,4,1,42.661575,21.096268,1,13,7,0.733333,...,0,1,0,0,1,1,0,0,0,0
2,7,15,4,1,31.411600,21.807230,1,13,5,0.200000,...,0,1,0,0,1,1,0,0,0,0
3,7,15,4,1,44.939400,21.362101,1,13,7,0.466667,...,0,1,0,0,1,1,0,0,0,0
4,7,15,4,1,13.325125,20.637705,0,14,5,0.933333,...,1,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155271,9,29,4,0,32.624625,31.877761,1,0,5,0.275862,...,0,1,0,0,1,1,0,0,0,0
155272,9,29,4,0,32.213225,31.043151,1,0,5,0.275862,...,0,1,0,0,1,1,0,0,0,0
155273,9,29,4,0,33.005775,33.266906,1,0,4,0.275862,...,0,1,0,0,1,1,0,0,0,0
155274,9,29,4,0,32.213225,33.154085,1,0,5,0.689655,...,0,1,0,0,1,1,0,0,0,0


## XGBRegressor

In [6]:
X=df_oh.drop(columns=['每坪價格','車位'])
y=df_oh['每坪價格']

In [7]:
# 調參前
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# 建立 XGBRegressor 模型
xgbr=xgb.XGBRegressor(tree_method="gpu_hist")

# 使用訓練資料訓練模型
xgbr.fit(X_train,y_train)

y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)

In [8]:
print(f'訓練集RMSE: {(mean_squared_error(y_train, y_train_pred,squared=False)):.3f},測試集: {(mean_squared_error(y_test, y_test_pred,squared=False)):.3f}')
print(f'訓練集MAE: {(mean_absolute_error(y_train, y_train_pred)):.3f},測試集: {(mean_absolute_error(y_test, y_test_pred)):.3f}')
print(f'訓練集R2: {(r2_score(y_train, y_train_pred)):.3f},測試集: {(r2_score(y_test, y_test_pred)):.3f}')

訓練集RMSE: 2.382,測試集: 2.610
訓練集MAE: 1.707,測試集: 1.832
訓練集R2: 0.834,測試集: 0.799


In [ ]:
importance=xgbr.feature_importances_
importance

In [ ]:
X_del=pd.DataFrame({'importance':xgbr.feature_importances_,'features':X.columns}).sort_values(by=['importance'],ascending=False)
mask_selected=X_del.iloc[0:20].features
X_selected=X_train[list(mask_selected)]
X_selected

In [ ]:
# grid 調參

params = {'learning_rate': [0.03, 0.05, 0.08, 0.1],
        'max_depth': [6, 8, 10, 12],
        'n_estimators' : [200, 500],
              }

xgbclf = xgb.XGBRegressor(tree_method='gpu_hist',enable_categorical=True)
clf = GridSearchCV(estimator=xgbclf,
                   param_grid=params,
                   n_jobs=4,
                   cv=3,
                   verbose=1)

clf.fit(X_train, y_train)

%time
print(clf.best_params_,clf.best_score_)

In [11]:
# 調參後
# X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3)

# scaler = preprocessing.StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)

xgbr=xgb.XGBRegressor(tree_method="gpu_hist",enable_categorical=True,n_jobs=-1,
                      colsample_bytree=0.5,
                      min_child_weight=5,
                      subsample=0.7,
                      max_depth=11,
                      n_estimators=500,
                      learning_rate=0.03)
# 使用訓練資料訓練模型
xgbr.fit(X_train,y_train)

# X_test = scaler.transform(X_test)
y_train_pred = xgbr.predict(X_train)
y_test_pred = xgbr.predict(X_test)

In [12]:
print(f'訓練集RMSE: {(mean_squared_error(y_train, y_train_pred,squared=False)):.3f},測試集: {(mean_squared_error(y_test, y_test_pred,squared=False)):.3f}')
print(f'訓練集MAE: {(mean_absolute_error(y_train, y_train_pred)):.3f},測試集: {(mean_absolute_error(y_test, y_test_pred)):.3f}')
print(f'訓練集R2: {(r2_score(y_train, y_train_pred)):.3f},測試集: {(r2_score(y_test, y_test_pred)):.3f}')

訓練集RMSE: 1.973,測試集: 2.433
訓練集MAE: 1.384,測試集: 1.654
訓練集R2: 0.886,測試集: 0.826


In [ ]:
kf =KFold(n_splits=5, shuffle=True, random_state=42)
sc=cross_val_score(model,X, y, cv= kf)

In [16]:
# Saving model
with open("model_XGBR_kaohsiung.pickle","wb") as f:
    pickle.dump(xgbr, f, protocol=pickle.HIGHEST_PROTOCOL)